#Decomposing Hamiltonian and Creating QUBO Formulation

##Import Necessary Packages

In [ ]:
# The code in [Decomposition Functions] and [Create Hamiltonian and Decompose] were largely contributions by Yujun Shen
import numpy as np
from numpy import linalg as LA
from numpy import kron    # for matrix math
import csv
import matplotlib.pyplot as plt
import time
import math


!pip install qiskit[optimization]
from qiskit_optimization.translators import from_ising
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit.opflow.primitive_ops import PauliSumOp


!pip install qiskit-aqua
from qiskit.optimization.applications.ising import stable_set
from qiskit.optimization.problems import QuadraticProgram



##Helper for printing matrices nicely

In [ ]:
def p_mat(mat):
	import numpy as np
	grid = np.matrix(mat)
	print(grid)

##Decomposition functions

In [ ]:
# helper functions defined from the two qubit decomposition, from michaelgoerz.net source linked
def HS(M1, M2):
    """Hilbert-Schmidt-Product of two matrices M1, M2"""
    return (np.dot(M1.conjugate().transpose(), M2)).trace()

def c2s(c):
    """Return a string representation of a complex number c"""
    if c == 0.0:
        return "0"
    if c.imag == 0:
        return "%g" % c.real
    elif c.real == 0:
        return "%gj" % c.imag
    else:
        return "%g+%gj" % (c.real, c.imag)

def decompose(H):
    """
    Decompose Hermitian matrix H into linear combination of Pauli matrices
    Works for both 4x4 (2 qubits) and 16x16 (4 qubits) cases
    """
    decomp_dict = {}    # to store results
    sx = np.array([[0, 1],  [ 1, 0]], dtype=np.complex128)    # define individual Pauli operator
    sy = np.array([[0, -1j],[1j, 0]], dtype=np.complex128)
    sz = np.array([[1, 0],  [0, -1]], dtype=np.complex128)
    id = np.array([[1, 0],  [ 0, 1]], dtype=np.complex128)
    S = [id, sx, sy, sz]
    labels = ['I', 'sigma_x', 'sigma_y', 'sigma_z']
    print("Tensor product decomposition of Hamiltonian:")
    
    if len(H) == 4: # 4x4 matrix for 2 qubits
        for i in range(4):
            for j in range(4):
                label = labels[i]+',' + labels[j]
                a_ij = 0.25 * HS(kron(S[i], S[j]), H)
                if abs(a_ij) >= 1E-10:
                    print("%s\t*\t( %s )" % (c2s(a_ij), label))   # save as string
                    decomp_dict[label] = float(a_ij)
                    
    elif len(H) == 16: # 16x16 matrix for 4 qubits
        for i in range(4):
            for j in range(4):
                for k in range(4):
                    for l in range(4):
                        label = labels[i]+',' + labels[j]+','+ labels[k] +',' + labels[l] 
                        a_ij = 1/16 * HS(kron(kron(S[i], S[j]),kron(S[k], S[l])), H)
                        if abs(a_ij) >= 1E-10:
                            print("%s\t*\t( %s )" % (c2s(a_ij), label))
                            decomp_dict[label] = float(a_ij)
    print("~~~ End ~~~")
    return decomp_dict      

##Create Hamiltonian and Decompose

In [ ]:
#paramaters given by Gunnar
sigma = math.sqrt(1/3)  #sigma squared is diffusion
delta = 1 #drift
m = 4 #number of qubits

#Hamiltonian mxm grid
h = [[0]*m for _ in range(m)]

for i in range(m):
    h[i][i] = (delta*(i+1)/m)
    if i+1 < m:
        h[i][i+1] = sigma**2
    if i-1 >= 0: 
        h[i][i-1] = sigma**2

p_mat(h)

Hamiltonian = h
min_eigenvalue = np.real(min(LA.eigvals(Hamiltonian)))
print("Check if Hermitian (all real eigenvals):", LA.eigvals(Hamiltonian))
print("The expected minimum eigensvalue is", round(min_eigenvalue,6), "\n")
Hamiltonian_dict = decompose(Hamiltonian)

[[0.25       0.33333333 0.         0.        ]
 [0.33333333 0.5        0.33333333 0.        ]
 [0.         0.33333333 0.75       0.33333333]
 [0.         0.         0.33333333 1.        ]]
Check if Hermitian (all real eigenvals): [-0.04224839  0.41437493  0.83562507  1.29224839]
The expected minimum eigensvalue is -0.042248 

Tensor product decomposition of Hamiltonian:
0.625	*	( I,I )
0.333333	*	( I,sigma_x )
-0.125	*	( I,sigma_z )
0.166667	*	( sigma_x,sigma_x )
0.166667	*	( sigma_y,sigma_y )
-0.25	*	( sigma_z,I )
~~~ End ~~~


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: ComplexWarning: Casting complex values to real discards the imaginary part


#Convert to Qubo

In [ ]:
#@title Using StackExchange [26146](https://quantumcomputing.stackexchange.com/questions/26146/convert-hamiltonian-to-ising-formulation-or-qubo)
# !pip install qiskit
# !pip install qiskit_optimization
# from qiskit_optimization.translators import from_ising
# from qiskit_optimization.converters import QuadraticProgramToQubo
# from qiskit.opflow.primitive_ops import PauliSumOp

# op   = PauliSumOp.from_list([("ZZ", 1), ("IZ", 2), ("Z", 3)]) # example operator
# # op = PauliSumOp.from_list([("II",0.625), ("IX",0.333333), ("IZ", -0.125), ("XX", 0.166667), ("YY", 0.166667), ("ZI", -0.25)])
# print(op)
# qp   = from_ising(op)
# conv = QuadraticProgramToQubo()
# qubo = conv.convert(qp)
# qubo

In [ ]:
#@title Using Stack Exchange [16069](https://quantumcomputing.stackexchange.com/questions/16069/qiskit-taking-a-qubo-matrix-into-qubit-op?noredirect=1&lq=1)
# !pip install qiskit-aqua
# from qiskit.optimization.applications.ising import stable_set
# from qiskit.optimization.problems import QuadraticProgram

qubitOp, offset = stable_set.get_operator(np.array(h))


print('Offset:', offset)
print('Ising Hamiltonian:')
print(qubitOp.print_details())

# # mapping Ising Hamiltonian to Quadratic Program
qp = QuadraticProgram()
qp.from_ising(qubitOp, offset)
qp


Offset: -0.5
Ising Hamiltonian:
IIZZ	(0.5+0j)
IZZI	(0.5+0j)
ZZII	(0.5+0j)
IIIZ	(-0.5+0j)
IIZI	(-1+0j)
IZII	(-1+0j)
ZIII	(-0.5+0j)



\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: [ 4 x_0*x_1 + 4 x_1*x_2 + 4 x_2*x_3 ]/2 -2
Subject To

Bounds
 0 <= x_0 <= 1
 0 <= x_1 <= 1
 0 <= x_2 <= 1
 0 <= x_3 <= 1

Binaries
 x_0 x_1 x_2 x_3
End